# Wikipedia Retriever 

In [1]:
from langchain_community.retrievers import WikipediaRetriever

In [2]:
retriever = WikipediaRetriever(top_k_results=3,lang='en')

In [3]:
query="One piece"
docs=retriever.invoke(query)

In [4]:
docs

[Document(metadata={'title': 'One Piece', 'summary': 'One Piece (stylized in all caps) is a Japanese manga series written and illustrated by Eiichiro Oda. It follows the adventures of Monkey D. Luffy and his crew, the Straw Hat Pirates, as he explores the Grand Line in search of the mythical treasure known as the "One Piece" to become the next King of the Pirates.\nThe manga has been serialized in Shueisha\'s shōnen manga magazine Weekly Shōnen Jump since July 1997, with its chapters compiled in 112 tankōbon volumes as of July 2025. It was licensed for an English language release in North America and the United Kingdom by Viz Media and in Australia by Madman Entertainment. Becoming a media franchise, it has been adapted into a festival film by Production I.G, and an anime series by Toei Animation, which premiered in October 1999. Additionally, Toei has developed 14 animated feature films and one original video animation (OVA). Several companies have developed various types of merchandi

In [5]:
for i,doc in enumerate(docs):
    print('Result',i+1,'\n')
    print(doc.page_content)
    



Result 1 

One Piece (stylized in all caps) is a Japanese manga series written and illustrated by Eiichiro Oda. It follows the adventures of Monkey D. Luffy and his crew, the Straw Hat Pirates, as he explores the Grand Line in search of the mythical treasure known as the "One Piece" to become the next King of the Pirates.
The manga has been serialized in Shueisha's shōnen manga magazine Weekly Shōnen Jump since July 1997, with its chapters compiled in 112 tankōbon volumes as of July 2025. It was licensed for an English language release in North America and the United Kingdom by Viz Media and in Australia by Madman Entertainment. Becoming a media franchise, it has been adapted into a festival film by Production I.G, and an anime series by Toei Animation, which premiered in October 1999. Additionally, Toei has developed 14 animated feature films and one original video animation (OVA). Several companies have developed various types of merchandising and media, such as a trading card game a

# Vector Store Retriever

In [6]:
%pwd

'c:\\Users\\ROSHAN\\OneDrive\\Desktop\\Vs Code\\Python\\lang\\Retrievers'

In [7]:
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from dotenv import load_dotenv


load_dotenv()

embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

c:\Users\ROSHAN\OneDrive\Desktop\Vs Code\Python\lang\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
docs = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [9]:
vector_store=Chroma(embedding_function=embedding_model,
                    persist_directory='../VectorStore/chroma_db',
                    collection_name='retriever'
                    )

C:\Users\ROSHAN\AppData\Local\Temp\ipykernel_3176\3214272978.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vector_store=Chroma(embedding_function=embedding_model,


In [10]:
vector_store.add_documents(documents=docs)

['c212fff4-b690-4f60-b5c7-a44af95c57e1',
 'dfbcdfcb-d11b-4ef2-adb0-cd175efdd39c',
 '02c418dc-7981-468c-9be4-73f6ec050892',
 '444f445e-a985-4193-80d8-7a9793032551']

In [11]:
# vector store retriever

retriever = vector_store.as_retriever(search_kwargs={'k':2})

query='what is  chroma used for'

In [12]:
res = retriever.invoke(query)
res

[Document(metadata={}, page_content='Chroma is a vector database optimized for LLM-based search.'),
 Document(metadata={}, page_content='Chroma is a vector database optimized for LLM-based search.')]

# "mmr" (maximum marginal relevance)

In [13]:
from langchain_community.vectorstores import FAISS

In [14]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [15]:
vector_store = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model, 
)

vector_store

In [16]:
# mmr retriever with lambda_mult=1 ...would behave same as default similarity_search...

retriever  = vector_store.as_retriever(search_type='mmr',search_kwargs={'k':3,'lambda_mult':1})


In [17]:
query = 'what is langchain'

res = retriever.invoke(query)

res

[Document(id='48705f76-4845-42d0-8879-d114365b1238', metadata={}, page_content='LangChain supports Chroma, FAISS, Pinecone, and more.'),
 Document(id='8a52ca50-b99b-4786-a635-fc3a14786a0f', metadata={}, page_content='LangChain is used to build LLM based applications.'),
 Document(id='b27df70e-f9c6-4f7f-acc3-cbc8564a7d5f', metadata={}, page_content='LangChain makes it easy to work with LLMs.')]

In [18]:
# mmr retriever with lambda_mult=0.5

retriever  = vector_store.as_retriever(search_type='mmr',search_kwargs={'k':3,'lambda_mult':0.5})


In [19]:
query = 'what is langchain'

res = retriever.invoke(query)

res

[Document(id='48705f76-4845-42d0-8879-d114365b1238', metadata={}, page_content='LangChain supports Chroma, FAISS, Pinecone, and more.'),
 Document(id='8a52ca50-b99b-4786-a635-fc3a14786a0f', metadata={}, page_content='LangChain is used to build LLM based applications.'),
 Document(id='2854e580-051a-45a8-ba74-36aa1de9993f', metadata={}, page_content='Embeddings are vector representations of text.')]

# Multiquery Retriever

In [32]:
from langchain_classic.retrievers import MultiQueryRetriever
from langchain_groq import ChatGroq

model = ChatGroq(model='openai/gpt-oss-120b')

In [21]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [22]:
vector_store=FAISS.from_documents(
    embedding=embedding_model,
    documents=all_docs,
)

In [33]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(search_kwargs={'k':4}),
    llm=model
)

In [34]:
query= 'How to improve energy levels and maintain balance?'

In [35]:
multiquery_res = multiquery_retriever.invoke(query)

In [36]:
multiquery_res

[Document(id='1d29d483-bead-40cf-8985-73d7c7b033b2', metadata={'source': 'H5'}, page_content='Drinking sufficient water throughout the day helps maintain metabolism and energy.'),
 Document(id='49f5f169-d137-41c5-a23e-04f6a5ebd6bd', metadata={'source': 'H2'}, page_content='Consuming leafy greens and fruits helps detox the body and improve longevity.'),
 Document(id='1703a7c0-ba45-485f-858c-9a3e81e7e6c7', metadata={'source': 'I1'}, page_content='The solar energy system in modern homes helps balance electricity demand.'),
 Document(id='e11a5a15-4cf2-4f33-8844-a99ccdfb5d8d', metadata={'source': 'H4'}, page_content='Mindfulness and controlled breathing lower cortisol and improve mental clarity.'),
 Document(id='e899a087-387c-454c-8648-ab83bfdaec87', metadata={'source': 'H1'}, page_content='Regular walking boosts heart health and can reduce symptoms of depression.'),
 Document(id='61279ba5-3e72-40fe-bcdc-e69f95d47be0', metadata={'source': 'H3'}, page_content='Deep sleep is crucial for cellu

# Contextual Compression Retriever

In [12]:
from langchain_groq import ChatGroq
from langchain.vectorstores import FAISS
from langchain_classic.retrievers import ContextualCompressionRetriever
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_classic.retrievers.document_compressors import LLMChainExtractor
from dotenv import load_dotenv

load_dotenv()

embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [6]:
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [7]:
vector_store = FAISS.from_documents(
    embedding=embedding_model,
    documents=docs
)
vector_store

In [10]:
base_retriever = vector_store.as_retriever(search_kwargs={'k':5})

In [13]:
llm=ChatGroq(model='openai/gpt-oss-120b')
compressor = LLMChainExtractor.from_llm(llm)

In [17]:
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor,
)  

In [15]:
query = 'what is photosynthesis'

res = compression_retriever.invoke(query)
res

[Document(metadata={'source': 'Doc1'}, page_content='Photosynthesis is the process by which green plants convert sunlight into energy.'),
 Document(metadata={'source': 'Doc2'}, page_content='The chlorophyll in plant cells captures sunlight during photosynthesis.'),
 Document(metadata={'source': 'Doc4'}, page_content='Photosynthesis does not occur in animal cells.')]

In [16]:
for i,val in enumerate(res):
    print(val,'\n')

page_content='Photosynthesis is the process by which green plants convert sunlight into energy.' metadata={'source': 'Doc1'} 

page_content='The chlorophyll in plant cells captures sunlight during photosynthesis.' metadata={'source': 'Doc2'} 

page_content='Photosynthesis does not occur in animal cells.' metadata={'source': 'Doc4'} 

